Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train and use lenet5 for character predication

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras

In [4]:
# Import traps: http://python-notes.curiousefficiency.org/en/latest/python_concepts/import_traps.html
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet5
from recognizer.networks import simple

Hyperparameters

In [5]:
batch_size = 256
epochs = 16

Setup dataset

In [6]:
emnist = EmnistDataset()

train_dataset = emnist.train_dataset.shuffle(1024).batch(batch_size)# emnist.train_dataset.shuffle(1024).repeat().batch(batch_size)
test_dataset = emnist.test_dataset.batch(batch_size)

Download path: /home/jupyter/line-reader/data/cache/datasets/matlab.zip
Processing data...
Balancing train dataset...
Target max number of images per class: 21635892
Dataset ready, with 1395864 training entries and 116323 test entries


> The tf.data API provides a software pipelining mechanism through the tf.data.Dataset.prefetch transformation, which can be used to decouple the time when data is produced from the time when data is consumed. In particular, the transformation uses a background thread and an internal buffer to prefetch elements from the input dataset ahead of the time they are requested. The number of elements to prefetch should be equal to (or possibly greater than) the number of batches consumed by a single training step. You could either manually tune this value, or set it to tf.data.experimental.AUTOTUNE which will prompt the tf.data runtime to tune the value dynamically at runtime.
~[https://www.tensorflow.org/alpha/guide/data_performance](https://www.tensorflow.org/alpha/guide/data_performance)

In [7]:
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [8]:
train_dataset

<PrefetchDataset shapes: ((None, 28, 28, 1), (None, 62)), types: (tf.float32, tf.int64)>

In [9]:
(x_train, y_train), = train_dataset.take(1)
input_shape = x_train[0].shape
print(f"x shape: {x_train.shape}, model input shape: {input_shape}")

x shape: (256, 28, 28, 1), model input shape: (28, 28, 1)


## Quickly fit one batch and that everything is working as expected check 

In [10]:
# (x_test, y_test), = test_dataset.take(1)

# # model = lenet5(input_shape=input_shape, number_of_classes=emnist.number_of_classes)
# model = simple(input_shape=input_shape, number_of_classes=emnist.number_of_classes)

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(x=x_train, y=y_train, epochs=5)
# model.evaluate(x_test, y_test)

# Model training

[Get started with TensorFlow 2.0 for experts](https://www.tensorflow.org/alpha/tutorials/quickstart/advanced)

In [11]:
model_checkpoints_path = Path("../recognizer/ckpts/character_model")
model_checkpoints_path.mkdir(parents=True, exist_ok=True)
model_save_path = Path("../recognizer/weights/character_model.h5")

In [12]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

# optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.RMSprop()

initial_learning_rate = 0.01
learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate_schedule)

In [13]:
# initial_learning_rate = 0.01
# learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)

# learning_rate_schedule.ExponentialDecay(0.05, 10, 0.96)

# optimizer = keras.optimizers.Adam(learning_rate=learning_rate_schedule)

# for step in range(100, 100000):
#     print(float(optimizer._decayed_lr(step)))

In [14]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [15]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [16]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [17]:
model = lenet5(input_shape=input_shape, number_of_classes=emnist.number_of_classes)
# model = simple(input_shape=input_shape, number_of_classes=emnist.number_of_classes)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [18]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, model_checkpoints_path, max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print(f"Restored from {manager.latest_checkpoint}")
else:
    print("Initializing from scratch.")

Initializing from scratch.


In [19]:
for epoch in range(1, epochs + 1):
    for images, labels in train_dataset:
        train_step(images, labels)
    
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)
    
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
        save_path = manager.save()
        print(f"💾 Saved checkpoint for step {int(ckpt.step)}: {save_path}")
        
    print(f"Epoch {epoch}, "\
#           f"Current learning rate: {optimizer._lr}, "\
          f"Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()*100}, "\
          f"Test Loss: {test_loss.result()}, Test Accuracy: {test_accuracy.result()*100}")

Epoch 1, Loss: 1.88480544090271, Accuracy: 77.29872131347656, Test Loss: 0.7967246174812317, Test Accuracy: 79.8509292602539
Epoch 2, Loss: 1.2853127717971802, Accuracy: 78.16015625, Test Loss: 0.7365587949752808, Test Accuracy: 79.0643310546875
Epoch 3, Loss: 1.0929853916168213, Accuracy: 78.29521179199219, Test Loss: 0.7173655033111572, Test Accuracy: 78.80269622802734
Epoch 4, Loss: 1.007307529449463, Accuracy: 78.19929504394531, Test Loss: 0.7089312076568604, Test Accuracy: 78.69424438476562
Epoch 5, Loss: 0.9563224911689758, Accuracy: 78.08367156982422, Test Loss: 0.745503306388855, Test Accuracy: 77.70501708984375
Epoch 6, Loss: 0.9271887540817261, Accuracy: 77.88587188720703, Test Loss: 0.7491323351860046, Test Accuracy: 77.522216796875
Epoch 7, Loss: 0.9108269810676575, Accuracy: 77.66666412353516, Test Loss: 0.7563627362251282, Test Accuracy: 77.39864349365234
Epoch 8, Loss: 0.8997725248336792, Accuracy: 77.4848861694336, Test Loss: 0.7983296513557434, Test Accuracy: 77.024856

In [20]:
# Next time, start from a clean slate
!rm -r {model_checkpoints_path}

# Save the model

In [21]:
model.save(model_save_path)

# Predict

In [22]:
# model.trainable = False